Generate synthetic audio (random frequency and amplitude)

In [2]:
import os
import numpy as np
from scipy.io.wavfile import write
from scipy.signal import sawtooth, square

# Audio properties
sample_rate = 8000  # Hz
duration = 2  # seconds
num_samples = sample_rate * duration

# Directory to save files
output_dir = "synthetic_audio"
classes = ['circle', 'heart', 'hexagon', 'square', 'star', 'triangle']

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


def generate_heart_wave(freq, amp, num_samples, sample_rate):
    """Amplitude modulated wave."""
    t = np.linspace(0, duration, num_samples, endpoint=False)
    carrier = np.sin(2 * np.pi * freq * t)
    modulator = 0.5 * (1 + np.sin(2 * np.pi * freq / 4 * t))  # Slow modulator
    return amp * carrier * modulator

def generate_hexagon_wave(freq, amp, num_samples, sample_rate):
    """Frequency modulated wave."""
    t = np.linspace(0, duration, num_samples, endpoint=False)
    mod_freq = freq / 2  # Modulation frequency
    mod = np.sin(2 * np.pi * mod_freq * t)
    return amp * np.sin(2 * np.pi * freq * t + mod)

def generate_square_wave(freq, amp, num_samples, sample_rate):
    """Square wave."""
    t = np.linspace(0, duration, num_samples, endpoint=False)
    return amp * square(2 * np.pi * freq * t)

def generate_star_wave(freq, amp, num_samples, sample_rate):
    """Sawtooth wave."""
    t = np.linspace(0, duration, num_samples, endpoint=False)
    return amp * sawtooth(2 * np.pi * freq * t)

def generate_circle_wave(freq, amp, num_samples, sample_rate):
    """Smooth oscillation: 'weeeeeeoo-weeeeeeoo'."""
    t = np.linspace(0, duration, num_samples, endpoint=False)
    mod_freq = 2  # Modulation frequency (oscillations per second)
    mod = np.sin(2 * np.pi * mod_freq * t)  # Smooth oscillation pattern
    carrier = np.sin(2 * np.pi * freq * (1 + 0.5 * mod) * t)  # Modulate the carrier
    return amp * carrier

def generate_triangle_wave(freq, amp, num_samples, sample_rate):
    """Stepped descending tone: 'weeeeeeeoooo...weeeeeeeoooo...'."""
    t = np.linspace(0, duration, num_samples, endpoint=False)
    stepped_freq = np.piecewise(
        t,
        [t < 0.5, (t >= 0.5) & (t < 1.0), (t >= 1.0) & (t < 1.5), t >= 1.5],
        [freq, freq * 0.75, freq * 0.5, freq * 0.25]
    )  # Step down frequency every 0.5 seconds
    return amp * np.sin(2 * np.pi * stepped_freq * t)


# Generate and save audio clips
wave_generators = {
    'circle': generate_circle_wave,
    'heart': generate_heart_wave,
    'hexagon': generate_hexagon_wave,
    'square': generate_square_wave,
    'star': generate_star_wave,
    'triangle': generate_triangle_wave
}

for class_name in classes:
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    for i in range(1, 201):
        # Generate random frequency and amplitude for variability
        freq = np.random.uniform(200, 800)  # Frequency between 200Hz and 800Hz
        amp = np.random.uniform(0.3, 0.9)  # Amplitude between 0.3 and 0.9

        # Generate waveform
        waveform = wave_generators[class_name](freq, amp, num_samples, sample_rate)

        # Add random noise to make each clip unique
        noise = np.random.normal(0, 0.02, num_samples)  # Small Gaussian noise
        waveform = np.clip(waveform + noise, -1.0, 1.0)  # Ensure values are within [-1, 1]

        # Save to file
        file_name = f"clip_{i:03d}.wav"
        file_path = os.path.join(class_dir, file_name)
        write(file_path, sample_rate, (waveform * 32767).astype(np.int16))  # Save as 16-bit PCM
        print(f"Saved: {file_path}")

print("All audio clips generated and saved.")


Saved: synthetic_audio/circle/clip_001.wav
Saved: synthetic_audio/circle/clip_002.wav
Saved: synthetic_audio/circle/clip_003.wav
Saved: synthetic_audio/circle/clip_004.wav
Saved: synthetic_audio/circle/clip_005.wav
Saved: synthetic_audio/circle/clip_006.wav
Saved: synthetic_audio/circle/clip_007.wav
Saved: synthetic_audio/circle/clip_008.wav
Saved: synthetic_audio/circle/clip_009.wav
Saved: synthetic_audio/circle/clip_010.wav
Saved: synthetic_audio/circle/clip_011.wav
Saved: synthetic_audio/circle/clip_012.wav
Saved: synthetic_audio/circle/clip_013.wav
Saved: synthetic_audio/circle/clip_014.wav
Saved: synthetic_audio/circle/clip_015.wav
Saved: synthetic_audio/circle/clip_016.wav
Saved: synthetic_audio/circle/clip_017.wav
Saved: synthetic_audio/circle/clip_018.wav
Saved: synthetic_audio/circle/clip_019.wav
Saved: synthetic_audio/circle/clip_020.wav
Saved: synthetic_audio/circle/clip_021.wav
Saved: synthetic_audio/circle/clip_022.wav
Saved: synthetic_audio/circle/clip_023.wav
Saved: synt

Generate synthetic audio (Fixed amplitude and noise leve, different frequency classes) - for analysis of message consistency within classes

In [2]:
import numpy as np
import os
from scipy.signal import square, sawtooth
from scipy.io.wavfile import write

# Constants
sample_rate = 16000
duration = 2  # seconds
num_samples = int(sample_rate * duration)
output_dir = "output_dataset"
classes = ['circle', 'heart', 'hexagon', 'square', 'star', 'triangle']

# Fixed amplitude and noise
FIXED_AMPLITUDE = 0.7
NOISE_LEVEL = 0.02

# Frequency class ranges
frequency_classes = {
    '0': (200, 300),
    '1': (400, 500),
    '2': (600, 800)
}

# --- Waveform Generators (same as before, assuming duration is global) ---

def generate_heart_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    carrier = np.sin(2 * np.pi * freq * t)
    modulator = 0.5 * (1 + np.sin(2 * np.pi * freq / 4 * t))
    return amp * carrier * modulator

def generate_hexagon_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    mod_freq = freq / 2
    mod = np.sin(2 * np.pi * mod_freq * t)
    return amp * np.sin(2 * np.pi * freq * t + mod)

def generate_square_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    return amp * square(2 * np.pi * freq * t)

def generate_star_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    return amp * sawtooth(2 * np.pi * freq * t)

def generate_circle_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    mod_freq = 2
    mod = np.sin(2 * np.pi * mod_freq * t)
    carrier = np.sin(2 * np.pi * freq * (1 + 0.5 * mod) * t)
    return amp * carrier

def generate_triangle_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    stepped_freq = np.piecewise(
        t,
        [t < 0.5, (t >= 0.5) & (t < 1.0), (t >= 1.0) & (t < 1.5), t >= 1.5],
        [freq, freq * 0.75, freq * 0.5, freq * 0.25]
    )
    return amp * np.sin(2 * np.pi * stepped_freq * t)

# Map class names to generator functions
wave_generators = {
    'circle': generate_circle_wave,
    'heart': generate_heart_wave,
    'hexagon': generate_hexagon_wave,
    'square': generate_square_wave,
    'star': generate_star_wave,
    'triangle': generate_triangle_wave
}

# --- Generate and Save Audio Clips ---

for class_name in classes:
    generator = wave_generators[class_name]

    for freq_class_label, (f_min, f_max) in frequency_classes.items():
        freq_dir = os.path.join(output_dir, class_name, freq_class_label)
        os.makedirs(freq_dir, exist_ok=True)

        for i in range(1, 41):
            freq = np.random.uniform(f_min, f_max)

            waveform = generator(freq, FIXED_AMPLITUDE)

            noise = np.random.normal(0, NOISE_LEVEL, num_samples)
            waveform = np.clip(waveform + noise, -1.0, 1.0)

            file_name = f"clip_{i:03d}.wav"
            file_path = os.path.join(freq_dir, file_name)
            write(file_path, sample_rate, (waveform * 32767).astype(np.int16))

        print(f"Saved 200 clips for {class_name} in frequency class {freq_class_label}")

print("All audio clips generated and saved.")


Saved 200 clips for circle in frequency class 0
Saved 200 clips for circle in frequency class 1
Saved 200 clips for circle in frequency class 2
Saved 200 clips for heart in frequency class 0
Saved 200 clips for heart in frequency class 1
Saved 200 clips for heart in frequency class 2
Saved 200 clips for hexagon in frequency class 0
Saved 200 clips for hexagon in frequency class 1
Saved 200 clips for hexagon in frequency class 2
Saved 200 clips for square in frequency class 0
Saved 200 clips for square in frequency class 1
Saved 200 clips for square in frequency class 2
Saved 200 clips for star in frequency class 0
Saved 200 clips for star in frequency class 1
Saved 200 clips for star in frequency class 2
Saved 200 clips for triangle in frequency class 0
Saved 200 clips for triangle in frequency class 1
Saved 200 clips for triangle in frequency class 2
All audio clips generated and saved.


Generate synthetic audio (Fixed frequency and noise leve, different amplitude classes) - for analysis of message consistency within classes

In [3]:
import numpy as np
import os
from scipy.signal import square, sawtooth
from scipy.io.wavfile import write

# Constants
sample_rate = 16000
duration = 2  # seconds
num_samples = int(sample_rate * duration)
output_dir = "output_dataset_amplitude"
classes = ['circle', 'heart', 'hexagon', 'square', 'star', 'triangle']

# Fixed frequency
FIXED_FREQUENCY = 440  # Standard A4 note, can be adjusted if needed
NOISE_LEVEL = 0.02

# Amplitude class ranges
amplitude_classes = {
    '0': (0.3, 0.4),
    '1': (0.5, 0.6),
    '2': (0.7, 0.9)
}

# --- Waveform Generators ---

def generate_heart_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    carrier = np.sin(2 * np.pi * freq * t)
    modulator = 0.5 * (1 + np.sin(2 * np.pi * freq / 4 * t))
    return amp * carrier * modulator

def generate_hexagon_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    mod_freq = freq / 2
    mod = np.sin(2 * np.pi * mod_freq * t)
    return amp * np.sin(2 * np.pi * freq * t + mod)

def generate_square_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    return amp * square(2 * np.pi * freq * t)

def generate_star_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    return amp * sawtooth(2 * np.pi * freq * t)

def generate_circle_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    mod_freq = 2
    mod = np.sin(2 * np.pi * mod_freq * t)
    carrier = np.sin(2 * np.pi * freq * (1 + 0.5 * mod) * t)
    return amp * carrier

def generate_triangle_wave(freq, amp):
    t = np.linspace(0, duration, num_samples, endpoint=False)
    stepped_freq = np.piecewise(
        t,
        [t < 0.5, (t >= 0.5) & (t < 1.0), (t >= 1.0) & (t < 1.5), t >= 1.5],
        [freq, freq * 0.75, freq * 0.5, freq * 0.25]
    )
    return amp * np.sin(2 * np.pi * stepped_freq * t)

# Map class names to generator functions
wave_generators = {
    'circle': generate_circle_wave,
    'heart': generate_heart_wave,
    'hexagon': generate_hexagon_wave,
    'square': generate_square_wave,
    'star': generate_star_wave,
    'triangle': generate_triangle_wave
}

# --- Generate and Save Audio Clips ---

for class_name in classes:
    generator = wave_generators[class_name]

    for amp_class_label, (a_min, a_max) in amplitude_classes.items():
        amp_dir = os.path.join(output_dir, class_name, amp_class_label)
        os.makedirs(amp_dir, exist_ok=True)

        for i in range(1, 41):
            amp = np.random.uniform(a_min, a_max)

            waveform = generator(FIXED_FREQUENCY, amp)

            noise = np.random.normal(0, NOISE_LEVEL, num_samples)
            waveform = np.clip(waveform + noise, -1.0, 1.0)

            file_name = f"clip_{i:03d}.wav"
            file_path = os.path.join(amp_dir, file_name)
            write(file_path, sample_rate, (waveform * 32767).astype(np.int16))

        print(f"Saved 40 clips for {class_name} in amplitude class {amp_class_label}")

print("All audio clips generated and saved.")


Saved 40 clips for circle in amplitude class 0
Saved 40 clips for circle in amplitude class 1
Saved 40 clips for circle in amplitude class 2
Saved 40 clips for heart in amplitude class 0
Saved 40 clips for heart in amplitude class 1
Saved 40 clips for heart in amplitude class 2
Saved 40 clips for hexagon in amplitude class 0
Saved 40 clips for hexagon in amplitude class 1
Saved 40 clips for hexagon in amplitude class 2
Saved 40 clips for square in amplitude class 0
Saved 40 clips for square in amplitude class 1
Saved 40 clips for square in amplitude class 2
Saved 40 clips for star in amplitude class 0
Saved 40 clips for star in amplitude class 1
Saved 40 clips for star in amplitude class 2
Saved 40 clips for triangle in amplitude class 0
Saved 40 clips for triangle in amplitude class 1
Saved 40 clips for triangle in amplitude class 2
All audio clips generated and saved.
